## Lecture 19: implementing image separation using independent component analysis and SVD
    Here we mix two images of the same size and then using ICA and SVD we try
    to separate them to their original versions.
    you can change the value of beta to make different mixings of the original images.

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
# Read the input images
S1 = imread("Sharbat_Gula.jpg")
S2 = imread("building.jpg")
# figure(); imshow(S1)
# figure(); imshow(S2)

# Reduce the resolution of image S2 by slicing to (400,254,3)
S2_r = S2[0:1536:3,0:2048:8,:]
S2 = S2_r[0:400,0:254,:]  

m,n,v = S1.shape

# mixing matrix 
beta = 2/5
A = array([[4/5,beta],[-2/5,3/4]])    

# Create mixed matrices
X1 = A[0,0]*S1 + A[0,1]*S2
X2 = A[1,0]*S1 + A[1,1]*S2

# do not have to reshape, but easy to appy the formula
x1 = reshape(X1,(m*n*v),'F')    
x2 = reshape(X2,(m*n*v),'F')

x1 = x1-mean(x1)
x2 = x2-mean(x2)

theta0: the angle of maximal/minimal variance obtained from analytical formula <br>
    $$\theta_0=\frac{1}{2}\arctan{\frac{-2\sum x1 x2}{\sum(x_2^2-x_1^2)}}$$ <br>
Ustar:  the first rotaion matrix <br> 
$$U^* = \begin{bmatrix}  \cos\theta_0 & \sin\theta_0 \\ -\sin\theta_0 & \cos\theta_0   \end{bmatrix}$$ <br>
sigma1($\sigma_1$): eigenvalue in maximum directional variance <br>
sigma2($\sigma_2$): eigenvalue in minimum directional variance <br>
Sigma_inv ($\sum^{-1}$): inverse of Sigma matrix of SVD analysis <br>
phi0 ($\phi_0$):  is obtained by minimizing Kurtosis through analytical formula

In [6]:
theta0 = 0.5*arctan(-2*sum(x1*x2)/sum(x2**2-x1**2))   
Ustar = array([[cos(theta0),sin(theta0)],[-sin(theta0),cos(theta0)]])  

sigma1 = sum((cos(theta0)*x1+sin(theta0)*x2)**2)    
sigma2 = sum((cos(theta0-pi/2)*x1+sin(theta0-pi/2)*x2)**2)   
Sigma_inv = array([[1/sqrt(sigma1), 0],[0, 1/sqrt(sigma2)]]) 

# Form new matrices x1bar and x2bar
x1bar = Sigma_inv[0,0]*(Ustar[0,0]*x1+Ustar[0,1]*x2)   
x2bar = Sigma_inv[1,1]*(Ustar[1,0]*x1+Ustar[1,1]*x2)

phi0 = (1/4)*arctan(-sum((2*x1bar**3*x2bar-2*x1bar*x2bar**3)/(x1bar**2+x2bar**2))/ \
                    sum((3*x1bar**2*x2bar**2-0.5*x1bar**4-0.5*x2bar**4)/\
                        (x1bar**2+x2bar**2)))
V = array([[cos(phi0),sin(phi0)],[-sin(phi0),cos(phi0)]])

s1 = V[0,0]*x1bar+V[0,1]*x2bar
s2 = V[1,0]*x1bar+V[1,1]*x2bar

# Transform s1 and s2 to have the range of 0-255 appropriate for rgb values
s1 = s1 - min(s1)         
s1 = s1*(255/max(s1))  

s2 = s2 - min(s2)
s2 = s2*(255/max(s2))

S1_reconstructed = reshape(s1,(m,n,v),'F')
S2_reconstructed = reshape(s2,(m,n,v),'F')

figure()
#original images
subplot(3,2,1), imshow(S1) 
subplot(3,2,2), imshow(S2)
# mixed images
subplot(3,2,3), imshow(uint8(X1))               
subplot(3,2,4), imshow(uint8(X2))
# separated images
subplot(3,2,5), imshow(uint8(S1_reconstructed))  
subplot(3,2,6), imshow(uint8(S2_reconstructed))

(<AxesSubplot:>, <matplotlib.image.AxesImage at 0x7f54f9849af0>)